In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import torchvision
from scipy import special as S
from scipy.stats import entropy
from scipy.spatial import distance
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import sys
sys.path.insert(1, '/home/kseuro/Kai/deeplearnphysics/pytorch/particle_generator/')

# Meitner Machine
import ewm
import utils
from dataloader import BottleLoader

In [ ]:
device = torch.device(1)

In [ ]:
exp_root = "/media/hdd1/kai/particle_generator/experiments/"

In [ ]:
mlp = False
if mlp:
    EWM_root = exp_root + 'ewm_models/mlp_ewm/'
else:
    EWM_root = exp_root + 'ewm_models/conv_ewm/'

In [ ]:
# Path to model weights
weights_dir = "weights/"

In [ ]:
EWM_paths = []
# EWM_root += 'evaluated_models/'
for path in os.listdir(EWM_root):
    EWM_paths.append(os.path.join(EWM_root, path))

print("-"*60)
for i in range(len(EWM_paths)):
    EWM_name = EWM_paths[i].split('/')[-1]
    print("\n Exp_{}:".format(str(i)), EWM_name, '\n')
    print("-"*60)

In [ ]:
EWM_dir = EWM_paths[1]

In [ ]:
# Create the full path to the EWM model
EWM_path = os.path.join(EWM_root, EWM_dir) + "/"
print("Path to EWM Generator Model set as: \n{}".format(EWM_path))

In [ ]:
config_csv = EWM_path + "config.csv"
config_df = pd.read_csv(config_csv, delimiter = ",")

In [ ]:
# Get the model architecture from config df
n_layers = int(config_df[config_df['Unnamed: 0'].str.contains("n_layers")==True]['0'].values.item())
n_hidden = int(config_df[config_df['Unnamed: 0'].str.contains("n_hidden")==True]['0'].values.item())
l_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("l_dim")==True]['0'].values.item())
im_size  = int(config_df[config_df['Unnamed: 0'].str.contains("dataset")==True]['0'].values.item())
z_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("z_dim")==True]['0'].values.item())
print("{} Layer model with {} hidden units per layer".format(n_layers, n_hidden))
print('Bottleneck dimension (unrolled): {}'.format(l_dim))

In [ ]:
# Model kwargs
fc_sizes = [n_hidden] * n_layers
ewm_kwargs = { 'z_dim':z_dim, 'fc_sizes':fc_sizes, 'n_out':l_dim }

In [ ]:
# Create generator on GPU
G = ewm.ewm_G(**ewm_kwargs).to(device)
G.weights_init()

In [ ]:
# Get checkpoint name(s)
EWM_checkpoint_path  = EWM_path + weights_dir
EWM_checkpoint_names = []
for file in os.listdir(EWM_checkpoint_path):
    EWM_checkpoint_names.append(os.path.join(EWM_checkpoint_path, file))

In [ ]:
print("-"*60)
for i in range(len(EWM_checkpoint_names)):
    name = EWM_checkpoint_names[i].split('/')[-1]
    print("\n {} :".format(str(i)), name, '\n')
    print("-"*60)

In [ ]:
# Select the checkpoint you want
EWM_checkpoint = EWM_checkpoint_names[0]

In [ ]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
checkpoint = torch.load(EWM_checkpoint)

In [ ]:
# Load the model's state dictionary
# Note: The IncompatibleKeys(missing_keys=[], unexpected_keys=[]) message indicates that
#       there were no problems in loading the state dictionary. Bit confusing...
G.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Put the model in evaluation mode
G.eval()

In [ ]:
config = {'ewm_target':'conv', 'num_workers':0, 'batch_size':1, 'shuffle':False, 'drop_last':False,
          'vec_root':'/media/hdd1/kai/particle_generator/code_vectors_test/', 'dataset':64, 'l_dim':l_dim}

In [ ]:
test_loader = utils.get_test_loader(config)

In [ ]:
g_vectors = []; t_vectors = []; epsilon = 1e-6

In [ ]:
for _, target_vec in enumerate(test_loader):
    
    # Get output from the generator model
    z_rand = torch.randn(config['batch_size'], 1, 100).to(device)
    code_vec = G(z_rand).view(-1).to(device)
    
    # Append vector from generator function to list
    g_vectors.append(code_vec.detach().cpu().numpy())
    
    # Append target vector to list
    t_vectors.append(target_vec.view(-1).detach().cpu().numpy())
    
    del code_vec
    del target_vec

[SciPi Entropy Function](http://scipy.github.io/devdocs/generated/scipy.stats.entropy.html)

In [ ]:
g_vector = np.array([g for g in g_vectors]).flatten() + epsilon
t_vector = np.array([t for t in t_vectors]).flatten() + epsilon